# **R基礎 その3**

このテキストでは、様々なパッケージを利用して、ネットワーク図・地図・系統樹などを描写してみます。

細かいコードの意味は大丈夫なので、Rを使うとどういったことが出来るのか、体験してみましょう。

今後使う機会が出て来たり気になる機能があれば、より細かい使い方等は自分で調べてみて下さい。

In [ ]:
# # network
# install.packages("igraph")
# install.packages("tidygraph")
# install.packages("ggraph")
# # phylogeny
# install.packages("ape")
# # map
# install.packages("maps")
# install.packages("mapdata")
# # map2
# install.packages("htmlwidgets")
# install.packages("magrittr")
# install.packages("leaflet")

In [ ]:
system("wget -q -O library.tgz https://github.com/slt666666/informatics_agri_1st/raw/main/source/_static/colab_notebook/library.tgz")
system("tar zxvf library.tgz")
.libPaths("library")

In [ ]:
library(igraph)
library(tidygraph)

A <- matrix(c(
  0,1,1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0, 
  1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0, 
  1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0, 
  1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0, 
  0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0, 
  1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0, 
  0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0, 
  0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,1, 
  0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1, 
  0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,
  0,1,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0, 
  1,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0, 
  1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0, 
  0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0, 
  0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0, 
  0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0, 
  0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0, 
  0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1, 
  0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1, 
  0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0),
  nrow = 20, ncol = 20, byrow = TRUE)

colnames(A) <- c(1:20)
rownames(A) <- c(1:20)

In [ ]:
g1 <- graph_from_adjacency_matrix(A)
g1_tbl <- as_tbl_graph(g1, direted = F)

library(ggraph)

# グラフ描写
plot_tbl_data <- function(tbl_data, title = "centrality"){
  tbl_data %>%  
    ggraph(layout = "kk") +
    geom_edge_link(alpha=0.8, colour = "lightgray") + 
    scale_edge_width(range = c(0.1,1)) +
    geom_node_point(aes( size = centrality)) +
    geom_node_label(aes(label = name),repel = TRUE)+
    ggtitle(title)
}

# 次数中心性
g1_tbl.deg <- g1_tbl %>% 
  mutate(centrality = centrality_degree())

plot_tbl_data(g1_tbl.deg, "centrality:degree")

In [ ]:
library(ape)
data(woodmouse)
wm.d<-dist.dna(woodmouse)
wm.hc<-hclust(wm.d,"average")
wm.phy<-as.phylo(wm.hc)
plot(wm.phy)

In [ ]:
library(maps)
library(mapdata)

map("japan", interior = FALSE)
map("japan", boundary = FALSE, lty = 2, add = TRUE)

In [ ]:
library(magrittr)
library(leaflet)

library(htmlwidgets)
m <- leaflet() %>% 
    addTiles() %>% 
    setView(lng=135,lat=35,zoom=7)
saveWidget(m, file="m.html")